# Ejercicio: Más métricas derivadas de matrices de confusión
En este ejercicio conoceremos diferentes métricas, utilizándolas para explicar los resultados obtenidos del modelo de clasificación binario que construimos en la unidad anterior.

## Visualización de datos
Utilizaremos una vez más el conjunto de datos con diferentes clases de objetos encontrados en la montaña:

In [1]:
import pandas
import numpy
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

#Let's have a look at the data
dataset

,size,roughness,color,motion,label
0,50.959361,1.318226,green,0.054290,tree
1,60.008521,0.554291,brown,0.000000,tree
2,20.530772,1.097752,white,1.380464,tree
3,28.092138,0.966482,grey,0.650528,tree
4,48.344211,0.799093,grey,0.000000,tree
...,...,...,...,...,...
2195,1.918175,1.182234,white,0.000000,animal
2196,1.000694,1.332152,black,4.041097,animal
2197,2.331485,0.734561,brown,0.961486,animal
2198,1.786560,0.707935,black,0.000000,animal


Recordemos que para utilizar el conjunto de datos anterior para la clasificación binaria, tenemos que añadir otra columna al conjunto de datos y establecerla en Verdadero cuando la etiqueta original sea excursionista y en Falso cuando no lo sea.

Añadamos entonces esa etiqueta, dividamos el conjunto de datos y volvamos a entrenar el modelo:

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Add a new label with true/false values to our dataset
dataset["is_hiker"] = dataset.label == "hiker"

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=1, shuffle=True)

# define a random forest model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used 
features = ["size", "roughness", "motion"]

# Train the model using the binary label
model.fit(train[features], train.is_hiker)

print("Model trained!")

Model trained!


Ahora podemos usar este modelo para predecir si los objetos en la nieve son excursionistas o no. Grafiquemos su matriz de confusión:

In [3]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the model's accuracy on the TEST set
actual = test.is_hiker
predictions = model.predict(test[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['True', 'False',]
unique_targets = sorted(list(test["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

In [4]:
# Let's also calculate some values that will be used throughout this exercise
# We already have actual values and corresponding predictions, defined above
correct = actual == predictions
tp = numpy.sum(correct & actual)
tn = numpy.sum(correct & numpy.logical_not(actual))
fp = numpy.sum(numpy.logical_not(correct) & actual)
fn = numpy.sum(numpy.logical_not(correct) & numpy.logical_not(actual))

print("TP - True Positives: ", tp)
print("TN - True Negatives: ", tn)
print("FP - False positives: ", fp)
print("FN - False negatives: ", fn)

TP - True Positives:  75
TN - True Negatives:  523
FP - False positives:  29
FN - False negatives:  33


Podemos utilizar los valores y la matriz anteriores para ayudarnos a entender otras métricas.

Cálculo de métricas
A partir de ahora vamos a examinar más detenidamente cada una de las siguientes métricas, cómo se calculan y cómo pueden ayudarnos a explicar nuestro modelo actual.

Precisión
Sensibilidad/Recuperación
Especificidad
Precisión
Tasa de falsos positivos
Recordemos primero algunos términos útiles:

TP = Verdaderos positivos: una etiqueta positiva se predice correctamente
TN = Verdaderos negativos: una etiqueta negativa se predice correctamente
FP = Falsos positivos: una etiqueta negativa se predice como positiva
FN = Falsos negativos: una etiqueta positiva se predice como negativa
## Precisión
La precisión es el número de predicciones correctas dividido por el número total de predicciones:

    precisión = (TP+TN) / número de muestras

Posiblemente sea la métrica más básica utilizada pero, como hemos visto, no es la más fiable cuando se utilizan conjuntos de datos desequilibrados. En codigo:

In [5]:
# Calculate accuracy
# len(actual) is the number of samples in the set that generated TP and TN
accuracy = (tp+tn) / len(actual) 

# print result as a percentage
print(f"Model accuracy is {accuracy:.2f}%")

Model accuracy is 0.91%


## Sensibilidad/Recuperación
Sensibilidad y Recuperación son nombres intercambiables para la misma métrica, que expresa la fracción de muestras predichas correctamente por un modelo:

    sensibilidad = recall = TP / (TP + FN)
Se trata de una métrica importante, que nos indica cuántas de todas las muestras realmente positivas se predicen correctamente como positivas.

En código:

In [6]:
# code for sensitivity/recall
sensitivity = recall = tp / (tp + fn)

# print result as a percentage
print(f"Model sensitivity/recall is {sensitivity:.2f}%")

Model sensitivity/recall is 0.69%


## Especificidad
La especificidad expresa la fracción de etiquetas negativas predichas correctamente sobre el número total de muestras negativas existentes:

    especificidad = TN / (TN + FP)
Nos indica cuántas de todas las muestras realmente negativas se predicen correctamente como negativas

Puede calcularse utilizando el código siguiente:

In [7]:
# Code for specificity
specificity = tn / (tn + fp)

# print result as a percentage
print(f"Model specificity is {specificity:.2f}%")

Model specificity is 0.95%


## Precisión
La precisión expresa la proporción de muestras positivas predichas correctamente sobre todas las predicciones positivas:

    precisión = TP / (TP + FP)
En otras palabras, indica cuántas de todas las predicciones positivas son realmente etiquetas positivas.

Puede calcularse utilizando el siguiente código:

In [8]:
# Code for precision

precision = tp / (tp + fp)

# print result as a percentage
print(f"Model precision is {precision:.2f}%")

Model precision is 0.72%


## Tasa de falsos positivos
La tasa de falsos positivos o FPR, es el número de predicciones positivas incorrectas dividido por el número total de muestras negativas:

    tasa_de_falsos_positivos = FP / (FP + TN)
De todos los negativos reales, ¿cuántos se clasificaron erróneamente como positivos?

En código:

In [9]:
# Code for false positive rate
false_positive_rate = fp / (fp + tn)

# print result as a percentage
print(f"Model false positive rate is {false_positive_rate:.2f}%")

Model false positive rate is 0.05%


Obsérvese que la suma de la especificidad y la tasa de falsos positivos debe ser siempre igual a 1.

Conclusión
Existen varias métricas diferentes que pueden ayudarnos a evaluar el rendimiento de un modelo, en el contexto de la calidad de sus predicciones.

Sin embargo, la elección de la métrica más adecuada depende principalmente de los datos y del problema que intentamos resolver.

Resumen
En esta unidad hemos tratado los siguientes temas

Cómo calcular las medidas más básicas utilizadas en la evaluación de modelos de clasificación: TP, FP, TN, FN.
Cómo utilizar las medidas anteriores para calcular métricas más significativas, como:
Precisión
Sensibilidad/Recuperación
Especificidad
Precisión
Tasa de falsos positivos
La elección de las métricas depende del conjunto de datos y del problema que intentamos resolver.